In [0]:
%run ./includes/configuration

In [0]:
%run ./includes/utilities

In [0]:
# Step 1: Creat RawDF DataFrame
rawDF = ingest_batch_raw(rawData)

In [0]:
# Step 2: Transform the Raw Data
transformedRawDF = transform_raw(rawDF)

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-1374048656977749> in <module> 
 1 # Step 2: Transform the Raw Data 
 ----> 2 transformedRawDF = transform_raw ( rawDF ) 

 <command-1374048656977777> in transform_raw (df) 
 1 def transform_raw ( df : DataFrame ) -> DataFrame : 
 2 return (
 ----> 3 df.select(
 4 lit ( "files.training.databricks.com" ) . alias ( "datasource" ) , 
 5 current_timestamp ( ) . alias ( "ingesttime" ) , 

 /databricks/spark/python/pyspark/sql/dataframe.py in select (self, *cols) 
 1824 [ Row ( name = 'Alice' , age = 12 ) , Row ( name = 'Bob' , age = 15 ) ] 
 1825 """
 -> 1826 jdf = self . _jdf . select ( self . _jcols ( * cols ) ) 
 1827 return DataFrame ( jdf , self . sql_ctx ) 
 1828 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : Column 'value' does not exist. Did you mean one of the following? [movie, _corrupt_record];
'Project [files.training.databricks.com AS datasource#272, current_timestamp() AS ingesttime#273, 'value, cast(current_timestamp() as date) AS p_ingestdate#274]
+- Relation [_corrupt_record#264,movie#265] json

In [0]:
#Step 3:
rawToBronzeWriter = batch_writer(dataframe=transformedRawDF, checkpoint=bronzeCheckpoint, name="write_raw_to_bronze",partition_column="p_ingestdate")
rawToBronzeWriter.save(bronzePath)

In [0]:
#Display the bronze table
bronzeDF = read_stream_delta(spark, bronzePath)
display(bronzeDF, streamName="display_bronze")

In [0]:
#Show running streams
for stream in spark.streams.active:
    print(stream.name)

In [0]:
dbutils.fs.rm(silverPath, recurse=True)

In [0]:
# move data to silver master table
silver_master_tracker = (bronzeDF
                         .select("movie",
                                 col("movie.BackdropUrl"), 
                                 col("movie.Budget"), 
                                 col("movie.CreatedBy"), 
                                 col("movie.CreatedDate").cast("date").alias("CreatedDate"), 
                                 col("movie.Id"), 
                                 col("movie.ImdbUrl"), 
                                 col("movie.OriginalLanguage"), 
                                 col("movie.Overview"), 
                                 col("movie.PosterUrl"), 
                                 col("movie.Price"), 
                                 col("movie.ReleaseDate"), 
                                 col("movie.Revenue"), 
                                 col("movie.RunTime"), 
                                 col("movie.Tagline"), 
                                 col("movie.Title"), 
                                 col("movie.TmdbUrl"), 
                                 col("movie.UpdatedBy"), 
                                 col("movie.UpdatedDate").cast("date").alias("UpdatedDate"), 
                                 col("movie.genres")
                                ))

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1374048656977792> in <module> 
 1 # move data to silver master table 
 ----> 2 silver_master_tracker = (bronzeDF
 3 .select("movie",
 4 col ( "movie.BackdropUrl" ) , 
 5 col ( "movie.Budget" ) , 

 NameError : name 'bronzeDF' is not defined

In [0]:
display(silver_master_tracker)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1374048656977859> in <module> 
 ----> 1 display ( silver_master_tracker ) 

 NameError : name 'silver_master_tracker' is not defined

In [0]:
'''from pyspark.sql.types import _parse_datatype_string
assert silver_master_tracker.schema == _parse_datatype_string(
    """
      movie STRING,
      BackdropUrl string,
      Budget double,
      CreatedBy string,
      CreatedDate DATE,
      Id long,
      ImdbUrl string,
      OriginalLanguage string,
      Overview string,
      PosterUrl string,
      Price double,
      ReleaseDate string,
      Revenue double,
      RunTime long,
      Tagline string,
      Title string,
      TmdbUrl string,
      UpdatedBy string,
      UpdatedDate DATE,
      genres array
      """), "Schemas do not match"
print("Assertion passed.")
'''

Out[139]: 'from pyspark.sql.types import _parse_datatype_string\nassert silver_master_tracker.schema == _parse_datatype_string(\n """\n movie STRING,\n BackdropUrl string,\n Budget double,\n CreatedBy string,\n CreatedDate DATE,\n Id long,\n ImdbUrl string,\n OriginalLanguage string,\n Overview string,\n PosterUrl string,\n Price double,\n ReleaseDate string,\n Revenue double,\n RunTime long,\n Tagline string,\n Title string,\n TmdbUrl string,\n UpdatedBy string,\n UpdatedDate DATE,\n genres array\n """), "Schemas do not match"\nprint("Assertion passed.")\n'

In [0]:
silver_master_tracker.count()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1374048656977861> in <module> 
 ----> 1 silver_master_tracker . count ( ) 

 NameError : name 'silver_master_tracker' is not defined

In [0]:
silver_master_tracker.dropna(how="all").count()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1374048656977862> in <module> 
 ----> 1 silver_master_tracker . dropna ( how = "all" ) . count ( ) 

 NameError : name 'silver_master_tracker' is not defined

In [0]:
# Split the silver dataFrame
silver_master_tracker_clean = silver_master_tracker.filter("runtime >= 0" and "budget >= 1000000")
silver_master_tracker_quarantine = silver_master_tracker.filter("budget < 1000000" or "runtime < 0")

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1374048656977863> in <module> 
 1 # Split the silver dataFrame 
 ----> 2 silver_master_tracker_clean = silver_master_tracker . filter ( "runtime >= 0" and "budget >= 1000000" ) 
 3 silver_master_tracker_quarantine = silver_master_tracker . filter ( "budget < 1000000" or "runtime < 0" ) 

 NameError : name 'silver_master_tracker' is not defined

In [0]:
silver_master_tracker_clean.count()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1374048656977864> in <module> 
 ----> 1 silver_master_tracker_clean . count ( ) 

 NameError : name 'silver_master_tracker_clean' is not defined

In [0]:
silver_master_tracker_quarantine.count()

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1374048656977865> in <module> 
 ----> 1 silver_master_tracker_quarantine . count ( ) 

 NameError : name 'silver_master_tracker_quarantine' is not defined

In [0]:
display(silver_master_tracker_quarantine)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1374048656977866> in <module> 
 ----> 1 display ( silver_master_tracker_quarantine ) 

 NameError : name 'silver_master_tracker_quarantine' is not defined

In [0]:
(silver_master_tracker_clean.select("BackdropUrl", "Budget", "CreatedBy", "CreatedDate", "Id", "ImdbUrl", "OriginalLanguage", "Overview", "PosterUrl", "Price", "ReleaseDate", "Revenue", "RunTime", "Tagline", "Title", "TmdbUrl", "UpdatedBy", "UpdatedDate", "genres")
    .write.format("delta")
    .mode("append")
    .save(silverPath))

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1374048656977867> in <module> 
 ----> 1 (silver_master_tracker_clean.select("BackdropUrl", "Budget", "CreatedBy", "CreatedDate", "Id", "ImdbUrl", "OriginalLanguage", "Overview", "PosterUrl", "Price", "ReleaseDate", "Revenue", "RunTime", "Tagline", "Title", "TmdbUrl", "UpdatedBy", "UpdatedDate", "genres")
 2 . write . format ( "delta" ) 
 3 . mode ( "append" ) 
 4 .save(silverPath))

 NameError : name 'silver_master_tracker_clean' is not defined

In [0]:
spark.sql(
    """
DROP TABLE IF EXISTS master_silver
"""
)

spark.sql(
    f"""
CREATE TABLE master_silver
USING DELTA
LOCATION "{silverPath}"
"""
)

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-1374048656977868> in <module> 
 5 )
 6 
 ----> 7 spark.sql(
 8 f"""
 9 CREATE TABLE master_silver

 /databricks/spark/python/pyspark/sql/session.py in sql (self, sqlQuery) 
 775 [ Row ( f1 = 1 , f2 = 'row1' ) , Row ( f1 = 2 , f2 = 'row2' ) , Row ( f1 = 3 , f2 = 'row3' ) ] 
 776 """
 --> 777 return DataFrame ( self . _jsparkSession . sql ( sqlQuery ) , self . _wrapped ) 
 778 
 779 def table ( self , tableName ) : 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : 
You are trying to create an external table `spark_catalog`.`default`.`master_silver`
from `dbfs:/dbfs/FileStore/tables/silver` using Databricks Delta, but the schema is not specified when the
input path is empty.

To learn more about Delta, see https://docs.microsoft.com/azure/databricks/delta/index

In [0]:
'''silverTable = spark.read.table("master_silver")
expected_schema = """
      BackdropUrl string,
      Budget double,
      CreatedBy string,
      CreatedDate DATE,
      Id long,
      ImdbUrl string,
      OriginalLanguage string,
      Overview string,
      PosterUrl string,
      Price double,
      ReleaseDate string,
      Revenue double,
      RunTime long,
      Tagline string,
      Title string,
      TmdbUrl string,
      UpdatedBy string,
      UpdatedDate DATE,
      genres array
"""
assert silverTable.schema == _parse_datatype_string(expected_schema), "Schemas do not match"
print("Assertion passed.")'''

Out[148]: 'silverTable = spark.read.table("master_silver")\nexpected_schema = """\n BackdropUrl string,\n Budget double,\n CreatedBy string,\n CreatedDate DATE,\n Id long,\n ImdbUrl string,\n OriginalLanguage string,\n Overview string,\n PosterUrl string,\n Price double,\n ReleaseDate string,\n Revenue double,\n RunTime long,\n Tagline string,\n Title string,\n TmdbUrl string,\n UpdatedBy string,\n UpdatedDate DATE,\n genres array\n"""\nassert silverTable.schema == _parse_datatype_string(expected_schema), "Schemas do not match"\nprint("Assertion passed.")'

In [0]:
# Step 1: Update Clean records
from delta.tables import DeltaTable

bronzeTable = DeltaTable.forPath(spark, bronzePath)
silverAugmented = silver_master_tracker_clean.withColumn("status", lit("loaded")).dropDuplicates()

update_match = "bronze.movie = clean.movie"
update = {"status": "clean.status"}

(
    bronzeTable.alias("bronze")
    .merge(silverAugmented.alias("clean"), update_match)
    .whenMatchedUpdate(set=update)
    .execute()
)

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-1374048656977870> in <module> 
 2 from delta . tables import DeltaTable
 3 
 ----> 4 bronzeTable = DeltaTable . forPath ( spark , bronzePath ) 
 5 silverAugmented = silver_master_tracker_clean . withColumn ( "status" , lit ( "loaded" ) ) . dropDuplicates ( ) 
 6 

 /databricks/spark/python/delta/tables.py in forPath (cls, sparkSession, path) 
 348 jsparkSession : "JavaObject" = sparkSession . _jsparkSession # type: ignore[attr-defined] 
 349 
 --> 350 jdt = jvm . io . delta . tables . DeltaTable . forPath ( jsparkSession , path ) 
 351 return DeltaTable ( sparkSession , jdt ) 
 352 

 /databricks/spark/python/lib/py4j-0.10.9.1-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 121 # Hide where the exception came from that shows a non-Pythonic 
 122 # JVM exception message. 
 --> 123 raise converted from None 
 124 else : 
 125 raise 

 AnalysisException : `/dbfs/FileStore/tables/bronze/` is not a Delta table.

In [0]:
# Step 2: Update Quarantined records
silverAugmented = silver_master_tracker_quarantine.withColumn("status", lit("quarantined")).dropDuplicates()

update_match = "bronze.movie = quarantine.movie"
update = {"status": "quarantine.status"}

(
    bronzeTable.alias("bronze")
    .merge(silverAugmented.alias("quarantine"), update_match)
    .whenMatchedUpdate(set=update)
    .execute()
)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1374048656977871> in <module> 
 1 # Step 2: Update Quarantined records 
 ----> 2 silverAugmented = silver_master_tracker_quarantine . withColumn ( "status" , lit ( "quarantined" ) ) . dropDuplicates ( ) 
 3 
 4 update_match = "bronze.movie = quarantine.movie" 
 5 update = { "status" : "quarantine.status" } 

 NameError : name 'silver_master_tracker_quarantine' is not defined